In [1]:
# Minicons Installation
# Introduction can be found https://kanishka.xyz/post/minicons-running-large-scale-behavioral-analyses-on-transformer-lms/
# Tutorial and code can be found https://github.com/kanishkamisra/minicons/blob/master/examples/surprisals.md
#!pip install minicons

from minicons import scorer
import pandas as pd
import numpy as np
import json
import csv
import re
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.formula.api as smf
import torch
from torch.utils.data import DataLoader
from transformers import GPT2LMHeadModel, GPT2TokenizerFast

/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/var/folders/vv/sc6b0f596n7dy9mbxlcnn1180000gq/T/ipykernel_70458/658913012.py:7: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [ ]:
model_path = 'colinglab/ConcreteGPT-124M-128ctx-Strict-Small-mixed_bins'

model = GPT2LMHeadModel.from_pretrained(model_path)
tokenizer = GPT2TokenizerFast.from_pretrained(model_path)
#model = AutoModelForCausalLM.from_pretrained(model_path, trust_remote_code = True)
#tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code = True)

# wrap with minicons scorer
lm_scorer = scorer.IncrementalLMScorer(model_path, device = 'cpu')
#lm_scorer = scorer.IncrementalLMScorer(model = model, tokenizer = tokenizer)

In [ ]:
def calculate_surprisal(sentence):
    '''
    Takes in a sentence, and outputs surprisal values for each word.
    '''
    
    input_sentence = sentence
    # token_score() function of Minicons takes in several parameters
    # if surprisal = True, the output value is surprisal instead of log likelihood
    # if base_two = True, the log likelihood will be in base 2
    # see Minicons documentations for details
    # score tokens
    token_surprisals = lm_scorer.token_score(input_sentence, surprisal = True, base_two = True)[0]

    '''
    # filter out special tokens (like <pad>)
    special_tokens = set(tokenizer.all_special_tokens)
    filtered = [
        (token, score)
        for (token, score) in token_surprisals
        if token not in special_tokens
    ]
    '''

   # expand tokens that contain multiple words
    expanded = []
    for token, score in token_surprisals:
    #for token, score in filtered:
        token = token.strip('Ġ') # remove space marker
        if token.count('Ġ') > 0:
            # multiple words inside
            words = token.split('Ġ')
            words = [word for word in words if word]  # remove empty strings
            for i, word in enumerate(words):
                expanded_token = word
                expanded.append((expanded_token, score / len(words)))  # split surprisal equally
        else:
            expanded.append((token, score))    
    #print(expanded)

    # use regex to split into words and punctuation
    words = re.findall(r'\w+|[^\w\s]', sentence)
    results = []

    token_pointer = 0

    for word in words:
        accumulated = ''
        word_surprisal = 0.0

        while token_pointer < len(expanded):
            token, surprisal = expanded[token_pointer]
            accumulated += token
            word_surprisal += surprisal
            token_pointer += 1

            if accumulated == word:
                results.append((word, word_surprisal))
                break
        else:
            results.append((word, word_surprisal))

    return results


sentence = 'I know that your friend gave a baguette to Mary last weekend.'
calculate_surprisal(sentence)

[('I', 0.0),
 ('know', 4.838698863983154),
 ('that', 3.7790958881378174),
 ('your', 7.082535743713379),
 ('friend', 7.652877330780029),
 ('gave', 9.867260932922363),
 ('a', 4.943260192871094),
 ('baguette', 32.5692253112793),
 ('to', 4.711269378662109),
 ('Mary', 14.36178970336914),
 ('last', 8.900314331054688),
 ('weekend', 6.6284589767456055),
 ('.', 0.8413198590278625)]

In [ ]:
def calculate_sentence_surprisal(word_surprisals):
    '''Returns total surprisal and average surprisal per word.'''
    
    scores = [score for word, score in word_surprisals]
    total = sum(scores)
    avg = total / len(scores)
    return total, avg

"""
def sum_region_surprisal(row, region_list):
    '''Sums surprisals of a specified sentence region, extracting the relevant part from sentence_surprisals using index information.'''

    sentence_surprisals = row['surprisals']
    
    # retrieve all sentence column(from 'prefix' till 'end')
    all_columns = row.index.tolist()  # get the list of column names
    prefix_index = all_columns.index('prefix')
    eos_index = all_columns.index('end')
    
    # reconstruct the full sentence from 'prefix' to 'end' (exclude NaN values)
    full_sentence = ' '.join([str(row[col]) if pd.notna(row.get(col)) else '' for col in all_columns[prefix_index:eos_index + 1]])
    
    # use regex to split the full sentence into words & punctuation
    sentence_units = re.findall(r'\w+|[^\w\s]', full_sentence)
    
    # get the start and end indices for the region_list within the full sentence
    first_region_column = region_list[0]
    first_region_start_index = all_columns.index(first_region_column)
    
    last_region_column = region_list[-1]
    last_region_end_index = all_columns.index(last_region_column)
    
    # extract the relevant slice of sentence_surprisals that corresponds to the region_list
    # calculate the starting and ending index for the region_list part
    start_index = sum([
        len(re.findall(r'\w+|[^\w\s]', str(row[col]))) for col in all_columns[prefix_index:first_region_start_index]
        if pd.notna(row[col]) 
    ])
    end_index = sum([
        len(re.findall(r'\w+|[^\w\s]', str(row[col]))) for col in all_columns[prefix_index:last_region_end_index + 1]
        if pd.notna(row[col]) 
    ])

    # extract the slice of sentence_surprisals corresponding to the region_list part
    relevant_surprisals = sentence_surprisals[start_index:end_index]
    region_surprisal = sum([score for token, score in relevant_surprisals])
    
    return region_surprisal
"""

def sum_region_surprisal(row, region_list, priority_region = None, normalize = False):
    '''Sums surprisals of a specified sentence region with optional priority region handling.'''

    sentence_surprisals = row['surprisals']
    
    # retrieve all sentence column(from 'prefix' till 'end')
    all_columns = row.index.tolist()  # get the list of column names
    prefix_index = all_columns.index('prefix')
    eos_index = all_columns.index('end')
    
    # reconstruct the full sentence from 'prefix' to 'end' (exclude NaN values)
    full_sentence = ' '.join([str(row[col]) if pd.notna(row.get(col)) else '' for col in all_columns[prefix_index:eos_index + 1]])
    
    # use regex to split the full sentence into words & punctuation
    sentence_units = re.findall(r'\w+|[^\w\s]', full_sentence)
    
    if priority_region:
        # check if priority_region has non-zero surprisal
        if pd.notna(row[priority_region]):
            # extract text for the priority_region
            priority_region_index = all_columns.index(priority_region)
            
            start_index = sum([
                len(re.findall(r'\w+|[^\w\s]', str(row[col]))) for col in all_columns[prefix_index:priority_region_index]
                if pd.notna(row[col]) 
            ])
            end_index = sum([
                len(re.findall(r'\w+|[^\w\s]', str(row[col]))) for col in all_columns[prefix_index:priority_region_index + 1]
                if pd.notna(row[col]) 
            ])
            
            # calculate the surprisal for the priority region
            relevant_surprisals = sentence_surprisals[start_index:end_index]
            priority_surprisal = sum([score for token, score in relevant_surprisals])
    
            # return priority region's surprisal score if it is non-zero
            if priority_surprisal != 0:
                return priority_surprisal
    
    # if priority region is not provided or its surprisal is zero, calculate sum of region_list surprisals

    # get the start and end indices for the region_list within the full sentence
    first_region_column = region_list[0]
    first_region_start_index = all_columns.index(first_region_column)
    
    last_region_column = region_list[-1]
    last_region_end_index = all_columns.index(last_region_column)
    
    # extract the relevant slice of sentence_surprisals that corresponds to the region_list
    # calculate the starting and ending index for the region_list part
    start_index = sum([
        len(re.findall(r'\w+|[^\w\s]', str(row[col]))) for col in all_columns[prefix_index:first_region_start_index]
        if pd.notna(row[col]) 
    ])
    end_index = sum([
        len(re.findall(r'\w+|[^\w\s]', str(row[col]))) for col in all_columns[prefix_index:last_region_end_index + 1]
        if pd.notna(row[col]) 
    ])

    # extract the slice of sentence_surprisals corresponding to the region_list part
    relevant_surprisals = sentence_surprisals[start_index:end_index]
    region_surprisal = sum([score for token, score in relevant_surprisals])
    
    if normalize == False:
        return region_surprisal
    elif normalize == True:
        return region_surprisal / len(relevant_surprisals)

In [ ]:
def split_ends(ends):
    '''Splits off sentence-final punctuation.'''
    
    match = re.match(r'^(.*?)([.!?])$', ends.strip())
    if match:
        return pd.Series([match.group(1), match.group(2)])
    else:
        return pd.Series([ends, ''])  # no end punctuation
    
def encode_wh_licensor(df):
    '''
    Adds a numeric column to the DataFrame:
    - 1 if licensor is a wh-licensor
    - -1 otherwise (licensor is "that" or absent)
    '''
    df['wh_numeric'] = df['filler'].apply(lambda x: 1 if x > 0 else -1)
    return df
    
def merge_sentence(row, syntactic_parts):
    '''Merges sentence columns back into full sentence.'''
    
    parts = [row[part] for part in syntactic_parts]         
    # filter out NaN or empty parts
    non_empty_parts = [str(part).strip() for part in parts if pd.notna(part) and str(part).strip() != '']
    # join with spaces and add end punctuation
    sentence = ' '.join(non_empty_parts) + (row['end'] if pd.notna(row['end']) else '')
    return sentence

def assign_grammaticality(row):
    ''' Determines grammaticality based on licensor and gap values.'''
    if row['filler'] > 0 and row['gap'] == 1:
        return 'gram+'
        #return 1
    elif row['filler'] == 0 and row['gap'] == 0:
        return 'gram+'
        #return 1
    elif row['filler'] > 0 and row['gap'] == 0:
        return 'gram-'
        #return 0
    elif row['filler'] == 0 and row['gap'] == 1:
        return 'gram-'
        #return 0

#### Statistical Analysis: Mixed-Effects Linear Regression Model

In [ ]:
from IPython.display import display
import warnings
from statsmodels.tools.sm_exceptions import ConvergenceWarning

def print_summary(result):
    '''
    Prints the descriptive header from a statsmodels MixedLMResults summary,
    and displays both the fixed-effects and random-effects components as pandas DataFrames.
    '''
    with warnings.catch_warnings():
        warnings.simplefilter("ignore", ConvergenceWarning)
        
        summary_str = str(result.summary())
        lines = summary_str.split('\n')

        # PART 1: HEADER
        print("\n=== MODEL SUMMARY ===")
        for line in lines:
            if 'Coef.' in line:
                break  # stop when reaching the coefficient table
            pairs = re.findall(r'(\S[^:]*):\s+([^\s][^:]*?)(?=\s{2,}|$)', line)
            for key, value in pairs:
                print(f"{key.strip()}: {value.strip()}")

        # PART 2a: FIXED EFFECTS TABLE
        fixed_table = []
        in_fixed_table = False
        random_start_idx = None

        for i, line in enumerate(lines):
            if 'Coef.' in line:
                in_fixed_table = True
                continue
            if in_fixed_table:
                if not line.strip() or set(line.strip()) in [{'='}, {'-'}]:
                    continue  # skip empty/separator lines
                parts = line.strip().split()
                if len(parts) >= 7:
                    param = parts[0]
                    row = parts[1:7]
                    fixed_table.append([param] + row)
                else:
                    random_start_idx = i
                    break

        if fixed_table:
            df_fixed = pd.DataFrame(fixed_table, columns = [
                'Parameter', 'Coef.', 'Std.Err.', 'z', 'P>|z|', '[0.025', '0.975]'
            ])
            print("\n=== FIXED EFFECTS COEFFICIENTS ===")
            display(df_fixed)
        else:
            print("\nNo fixed-effects data found.")

        # PART 2b: RANDOM EFFECTS / VARIANCE COMPONENTS TABLE
        random_table = []
        if random_start_idx:
            for line in lines[random_start_idx:]:
                if not line.strip() or set(line.strip()) in [{'='}, {'-'}]:
                    continue
                parts = line.strip().split()
                # Look for last token ending in Var or Cov
                varcov_idx = None
                for j in reversed(range(len(parts))):
                    if parts[j].endswith('Var') or parts[j].endswith('Cov'):
                        varcov_idx = j
                        break
                if varcov_idx is not None:
                    param = ' '.join(parts[:varcov_idx + 1])
                    numeric_parts = parts[varcov_idx + 1:]
                    if len(numeric_parts) == 2:
                        val1, val2 = numeric_parts
                    elif len(numeric_parts) == 1:
                        val1, val2 = numeric_parts[0], ''
                    else:
                        val1, val2 = '', ''
                    random_table.append([param, val1, val2])

        if random_table:
            df_random = pd.DataFrame(random_table, columns = ['Parameter', 'Coef.', 'Std.Err.'])
            print("\n=== RANDOM EFFECTS / VARIANCE-COVARIANCE ===")
            display(df_random)
        else:
            print("\nNo random-effects data found.")


In [ ]:
# columns: item_id (indicates sentence set), wh_licensor (0/1), gap (0/1), island_type, surprisal

warnings.simplefilter("ignore", ConvergenceWarning)

def mixed_effects_linear_regression(df, iv_list, surprisal, label):
    '''
    Fits mixed-effects model and extracts wh-licensing interaction.
    '''
        
    for field in iv_list:
        if field == 'wh_numeric' or 'gap_numeric' or 'gap_distance_length':
            pass
        else:
            df[field] = df[field].astype('category')

    interaction_terms = ' * '.join(iv_list)
    random_effects = ' + '.join(iv_list)

    model = smf.mixedlm(
        f"{surprisal} ~ {interaction_terms}",
        df,
        groups = df["item_id"],
        #re_formula = f"~{random_effects}"
        re_formula = "1"
        )

    result = model.fit()
    #interaction_coef = result.params.get('wh_licensor[T.1]:gap[T.1]', None)

    print(f"\n=== {label.upper()} ===")
    #print(result.summary())
    print_summary(result)

    return result

#interaction = mixed_effects_linear_regression(df, "construction_type") # label name to be changed according to construction type

#### Gap Distance - Categorical/Continuous

In [ ]:
sentence_df = pd.read_csv('test_sentences/Gap Distance.csv')
sentence_df['gap_distance'] = sentence_df['gap_distance'].fillna('null')
#sentence_df['gap_distance'].dropna(inplace = True)
#sentence_df = sentence_df[sentence_df['gap_distance'].replace(['nan', 'NaN'], np.nan).notna()]

In [ ]:
sentence_df[['temp_mod', 'end']] = sentence_df['temp_mod'].apply(split_ends)
sentence_df = encode_wh_licensor(sentence_df)
sentence_df['gap_numeric'] = sentence_df['gap'].replace(0, -1)
syntactic_parts = ['prefix', 'licensor', 'subj', 'modifier', 'verb', 'object', 'prep', 'goal', 'temp_mod']
sentence_df['sentence'] = sentence_df.apply(lambda x: merge_sentence(x, syntactic_parts), axis = 1)
sentence_df['surprisals'] = sentence_df['sentence'].apply(calculate_surprisal)
sentence_df['gap_distance_length'] = sentence_df['modifier'].apply(lambda x: len(x) if pd.notna(x) else 0)
sentence_df['grammaticality'] = sentence_df.apply(assign_grammaticality, axis = 1)

Gap Position == Object

In [ ]:
# modifier: short_mod, med_mod, long_mod

In [ ]:
object_df = sentence_df.copy()
#object_df['region_surprisal'] = object_df.apply(lambda x: sum_region_surprisal(x, ['prep', 'goal']), axis = 1)
object_df['region_surprisal'] = object_df.apply(lambda x: sum_region_surprisal(x, ['prep', 'goal'], 'object'), axis = 1)

# filter for object gap
objectgap_df = object_df[object_df['gap_position'] == 'DO']

In [ ]:
# flip test: gap-
no_gap_df = objectgap_df[objectgap_df['gap_numeric'] == -1]
iv_list = ['wh_numeric']
mixed_effects_linear_regression(no_gap_df, iv_list, 'region_surprisal', 'object gap position: gap-')


=== OBJECT GAP POSITION: GAP- ===

=== MODEL SUMMARY ===
:            MixedLM Dependent Variable: region_surprisal
No. Observations: 160
Method: REML
No. Groups: 20
Scale: 47.2481
Min. group size: 8
Log-Likelihood: -553.8490
Max. group size: 8
Converged: Yes
Mean group size: 8.0

=== FIXED EFFECTS COEFFICIENTS ===


,Parameter,Coef.,Std.Err.,z,P>|z|,[0.025,0.975]
0,Intercept,24.865,1.557,15.967,0.000,21.813,27.917
1,wh_numeric,0.595,0.543,1.095,0.274,-0.470,1.660



=== RANDOM EFFECTS / VARIANCE-COVARIANCE ===


,Parameter,Coef.,Std.Err.
0,Group Var,42.598,2.441


In [ ]:
# flip test: gap+
gap_df = objectgap_df[objectgap_df['gap_numeric'] == 1]
iv_list = ['wh_numeric']
mixed_effects_linear_regression(gap_df, iv_list, 'region_surprisal', 'object gap position: gap+')


=== OBJECT GAP POSITION: GAP+ ===

=== MODEL SUMMARY ===
:            MixedLM Dependent Variable: region_surprisal
No. Observations: 160
Method: REML
No. Groups: 20
Scale: 50.8276
Min. group size: 8
Log-Likelihood: -556.3436
Max. group size: 8
Converged: Yes
Mean group size: 8.0

=== FIXED EFFECTS COEFFICIENTS ===


,Parameter,Coef.,Std.Err.,z,P>|z|,[0.025,0.975]
0,Intercept,19.787,1.360,14.554,0.000,17.122,22.451
1,wh_numeric,-0.103,0.564,-0.183,0.855,-1.208,1.001



=== RANDOM EFFECTS / VARIANCE-COVARIANCE ===


,Parameter,Coef.,Std.Err.
0,Group Var,30.612,1.794


Gap Postion == PP/Goal

In [ ]:
pp_df = sentence_df.copy()
#pp_df['region_surprisal'] = pp_df.apply(lambda x: sum_region_surprisal(x, ['temp_mod']), axis = 1)
pp_df['region_surprisal'] = pp_df.apply(lambda x: sum_region_surprisal(x, ['temp_mod'], 'goal'), axis = 1)

# filter for PP gap
ppgap_df = pp_df[pp_df['gap_position'] == 'IO']

In [ ]:
# flip test: gap-
no_gap_df = ppgap_df[ppgap_df['gap_numeric'] == -1]
iv_list = ['wh_numeric']
mixed_effects_linear_regression(no_gap_df, iv_list, 'region_surprisal', 'pp gap position: gap-')


=== PP GAP POSITION: GAP- ===

=== MODEL SUMMARY ===
:            MixedLM Dependent Variable: region_surprisal
No. Observations: 160
Method: REML
No. Groups: 20
Scale: 38.2085
Min. group size: 8
Log-Likelihood: -535.7466
Max. group size: 8
Converged: Yes
Mean group size: 8.0

=== FIXED EFFECTS COEFFICIENTS ===


,Parameter,Coef.,Std.Err.,z,P>|z|,[0.025,0.975]
0,Intercept,16.167,1.306,12.379,0.000,13.608,18.727
1,wh_numeric,0.072,0.489,0.147,0.883,-0.886,1.030



=== RANDOM EFFECTS / VARIANCE-COVARIANCE ===


,Parameter,Coef.,Std.Err.
0,Group Var,29.337,1.909


In [ ]:
# flip test: gap+
gap_df = ppgap_df[ppgap_df['gap_numeric'] == 1]
iv_list = ['wh_numeric']
mixed_effects_linear_regression(gap_df, iv_list, 'region_surprisal', 'pp gap position: gap+')


=== PP GAP POSITION: GAP+ ===

=== MODEL SUMMARY ===
:            MixedLM Dependent Variable: region_surprisal
No. Observations: 160
Method: REML
No. Groups: 20
Scale: 56.9841
Min. group size: 8
Log-Likelihood: -571.8688
Max. group size: 8
Converged: Yes
Mean group size: 8.0

=== FIXED EFFECTS COEFFICIENTS ===


,Parameter,Coef.,Std.Err.,z,P>|z|,[0.025,0.975]
0,Intercept,24.357,2.026,12.023,0.000,20.387,28.328
1,wh_numeric,-0.073,0.597,-0.123,0.902,-1.243,1.096



=== RANDOM EFFECTS / VARIANCE-COVARIANCE ===


,Parameter,Coef.,Std.Err.
0,Group Var,74.964,3.761


Embedded Clause

In [ ]:
embed = ['subj', 'modifier', 'verb', 'object', 'prep', 'goal', 'temp_mod', 'end']
sentence_df['embed_surprisal'] = sentence_df.apply(lambda x: sum_region_surprisal(x, embed, normalize = True), axis = 1)

Embedded Clause - Object Gap Position

In [ ]:
# filter for object gap
embed_objectgap_df = sentence_df[sentence_df['gap_position'] == 'DO']

In [ ]:
# flip test: gap-
no_gap_df = embed_objectgap_df[embed_objectgap_df['gap_numeric'] == -1]
iv_list = ['wh_numeric']
mixed_effects_linear_regression(no_gap_df, iv_list, 'embed_surprisal', 'object gap position: gap-')


=== OBJECT GAP POSITION: GAP- ===

=== MODEL SUMMARY ===
:            MixedLM Dependent Variable: embed_surprisal
No. Observations: 160
Method: REML
No. Groups: 20
Scale: 0.4265
Min. group size: 8
Log-Likelihood: -182.9174
Max. group size: 8
Converged: Yes
Mean group size: 8.0

=== FIXED EFFECTS COEFFICIENTS ===


,Parameter,Coef.,Std.Err.,z,P>|z|,[0.025,0.975]
0,Intercept,8.650,0.156,55.565,0.000,8.345,8.955
1,wh_numeric,0.166,0.052,3.218,0.001,0.065,0.267



=== RANDOM EFFECTS / VARIANCE-COVARIANCE ===


,Parameter,Coef.,Std.Err.
0,Group Var,0.431,0.257


In [ ]:
# flip test: gap+
gap_df = embed_objectgap_df[embed_objectgap_df['gap_numeric'] == 1]
iv_list = ['wh_numeric']
mixed_effects_linear_regression(gap_df, iv_list, 'embed_surprisal', 'object gap position: gap+')


=== OBJECT GAP POSITION: GAP+ ===

=== MODEL SUMMARY ===
:            MixedLM Dependent Variable: embed_surprisal
No. Observations: 160
Method: REML
No. Groups: 20
Scale: 0.6597
Min. group size: 8
Log-Likelihood: -215.6391
Max. group size: 8
Converged: Yes
Mean group size: 8.0

=== FIXED EFFECTS COEFFICIENTS ===


,Parameter,Coef.,Std.Err.,z,P>|z|,[0.025,0.975]
0,Intercept,8.417,0.177,47.638,0.000,8.070,8.763
1,wh_numeric,0.106,0.064,1.644,0.100,-0.020,0.231



=== RANDOM EFFECTS / VARIANCE-COVARIANCE ===


,Parameter,Coef.,Std.Err.
0,Group Var,0.542,0.266


In [ ]:
# division by grammaticality
iv_list = ['grammaticality']
mixed_effects_linear_regression(embed_objectgap_df, iv_list, 'embed_surprisal', 'object gap position: division by grammaticality')


=== OBJECT GAP POSITION: DIVISION BY GRAMMATICALITY ===

=== MODEL SUMMARY ===
Model: MixedLM
Dependent Variable: embed_surprisal
No. Observations: 320
Method: REML
No. Groups: 20
Scale: 0.5917
Min. group size: 16
Log-Likelihood: -397.6622
Max. group size: 16
Converged: Yes
Mean group size: 16.0

=== FIXED EFFECTS COEFFICIENTS ===


,Parameter,Coef.,Std.Err.,z,P>|z|,[0.025,0.975]
0,Intercept,8.503,0.164,51.755,0.000,8.181,8.825
1,grammaticality[T.gram-],0.061,0.086,0.705,0.481,-0.108,0.229



=== RANDOM EFFECTS / VARIANCE-COVARIANCE ===


,Parameter,Coef.,Std.Err.
0,Group Var,0.466,0.219


Embedded Clause - PP/Goal Gap Position

In [ ]:
# filter for PP gap
embed_ppgap_df = sentence_df[sentence_df['gap_position'] == 'IO']

In [ ]:
# flip test: gap-
no_gap_df = embed_ppgap_df[embed_ppgap_df['gap_numeric'] == -1]
iv_list = ['wh_numeric']
mixed_effects_linear_regression(no_gap_df, iv_list, 'embed_surprisal', 'pp gap position: gap-')


=== PP GAP POSITION: GAP- ===

=== MODEL SUMMARY ===
:            MixedLM Dependent Variable: embed_surprisal
No. Observations: 160
Method: REML
No. Groups: 20
Scale: 0.4167
Min. group size: 8
Log-Likelihood: -181.6028
Max. group size: 8
Converged: Yes
Mean group size: 8.0

=== FIXED EFFECTS COEFFICIENTS ===


,Parameter,Coef.,Std.Err.,z,P>|z|,[0.025,0.975]
0,Intercept,8.597,0.158,54.360,0.000,8.287,8.907
1,wh_numeric,0.113,0.051,2.218,0.027,0.013,0.213



=== RANDOM EFFECTS / VARIANCE-COVARIANCE ===


,Parameter,Coef.,Std.Err.
0,Group Var,0.448,0.268


In [ ]:
# flip test: gap+
gap_df = embed_ppgap_df[embed_ppgap_df['gap_numeric'] == 1]
iv_list = ['wh_numeric']
mixed_effects_linear_regression(gap_df, iv_list, 'embed_surprisal', 'pp gap position: gap+')


=== PP GAP POSITION: GAP+ ===

=== MODEL SUMMARY ===
:            MixedLM Dependent Variable: embed_surprisal
No. Observations: 160
Method: REML
No. Groups: 20
Scale: 0.4455
Min. group size: 8
Log-Likelihood: -184.6307
Max. group size: 8
Converged: Yes
Mean group size: 8.0

=== FIXED EFFECTS COEFFICIENTS ===


,Parameter,Coef.,Std.Err.,z,P>|z|,[0.025,0.975]
0,Intercept,8.961,0.145,61.697,0.000,8.676,9.246
1,wh_numeric,0.108,0.053,2.049,0.040,0.005,0.212



=== RANDOM EFFECTS / VARIANCE-COVARIANCE ===


,Parameter,Coef.,Std.Err.
0,Group Var,0.366,0.219


In [ ]:
# division by grammaticality
iv_list = ['grammaticality']
mixed_effects_linear_regression(embed_ppgap_df, iv_list, 'embed_surprisal', 'pp gap position: division by grammaticality')


=== PP GAP POSITION: DIVISION BY GRAMMATICALITY ===

=== MODEL SUMMARY ===
Model: MixedLM
Dependent Variable: embed_surprisal
No. Observations: 320
Method: REML
No. Groups: 20
Scale: 0.4846
Min. group size: 16
Log-Likelihood: -366.2289
Max. group size: 16
Converged: Yes
Mean group size: 16.0

=== FIXED EFFECTS COEFFICIENTS ===


,Parameter,Coef.,Std.Err.,z,P>|z|,[0.025,0.975]
0,Intercept,8.776,0.151,58.071,0.000,8.480,9.072
1,grammaticality[T.gram-],0.005,0.078,0.065,0.948,-0.147,0.158



=== RANDOM EFFECTS / VARIANCE-COVARIANCE ===


,Parameter,Coef.,Std.Err.
0,Group Var,0.396,0.205


#### Double Gaps

In [ ]:
sentence_df = pd.read_csv('test_sentences/Double Gaps.csv')

In [ ]:
def assign_grammaticality_double_gaps(row):
    ''' Determines grammaticality based on licensor and gap values for the double gap construction.'''
    if row['filler'] == 1 and row['gap_count'] == 1:
        return 'gram+'
    elif row['filler'] == 0 and row['gap_count'] == 0:
        return 'gram+'
    elif row['filler'] == 1 and row['gap_count'] == 0:
        return 'gram-'
    elif row['filler'] == 0 and row['gap_count'] == 1:
        return 'gram-'
    elif row['filler'] == 0 and row['gap_count'] == 2:
        return 'gram-'
    elif row['filler'] == 1 and row['gap_count'] == 2:
        return 'gram-'

In [ ]:
sentence_df[['modifier', 'end']] = sentence_df['modifier'].apply(split_ends)
sentence_df = encode_wh_licensor(sentence_df)
sentence_df['gap_numeric'] = sentence_df['gap'].replace(0, -1)
syntactic_parts = ['prefix', 'licensor', 'subj', 'verb', 'object', 'modifier']
sentence_df['sentence'] = sentence_df.apply(lambda x: merge_sentence(x, syntactic_parts), axis = 1)
sentence_df['surprisals'] = sentence_df['sentence'].apply(calculate_surprisal)
sentence_df['gap_count'] = sentence_df['subject_gap'] + sentence_df['object_gap']
sentence_df['grammaticality'] = sentence_df.apply(assign_grammaticality_double_gaps, axis = 1)

Embedded Clause

In [ ]:
embed = ['subj', 'object', 'verb', 'modifier', 'end']
sentence_df['embed_surprisal'] = sentence_df.apply(lambda x: sum_region_surprisal(x, embed, normalize = True), axis = 1)

In [ ]:
# flip test: gap = 0
gap_0 = sentence_df[sentence_df['gap_count'] == 0]
iv_list = ['wh_numeric']
mixed_effects_linear_regression(gap_0, iv_list, 'embed_surprisal', 'double gap embed: gap = 0')


=== DOUBLE GAP EMBED: GAP = 0 ===

=== MODEL SUMMARY ===
:            MixedLM Dependent Variable: embed_surprisal
No. Observations: 40
Method: REML
No. Groups: 20
Scale: 0.0736
Min. group size: 2
Log-Likelihood: -40.0819
Max. group size: 2
Converged: Yes
Mean group size: 2.0

=== FIXED EFFECTS COEFFICIENTS ===


,Parameter,Coef.,Std.Err.,z,P>|z|,[0.025,0.975]
0,Intercept,7.157,0.232,30.889,0.000,6.703,7.612
1,wh_numeric,0.156,0.043,3.643,0.000,0.072,0.240



=== RANDOM EFFECTS / VARIANCE-COVARIANCE ===


,Parameter,Coef.,Std.Err.
0,Group Var,1.037,1.816


In [ ]:
# flip test: gap = 1
gap_1 = sentence_df[sentence_df['gap_count'] == 1]
iv_list = ['wh_numeric']
mixed_effects_linear_regression(gap_1, iv_list, 'embed_surprisal', 'double gap embed: gap = 1')


=== DOUBLE GAP EMBED: GAP = 1 ===

=== MODEL SUMMARY ===
:            MixedLM Dependent Variable: embed_surprisal
No. Observations: 80
Method: REML
No. Groups: 20
Scale: 0.3416
Min. group size: 4
Log-Likelihood: -95.9450
Max. group size: 4
Converged: Yes
Mean group size: 4.0

=== FIXED EFFECTS COEFFICIENTS ===


,Parameter,Coef.,Std.Err.,z,P>|z|,[0.025,0.975]
0,Intercept,7.563,0.217,34.898,0.000,7.138,7.988
1,wh_numeric,-0.086,0.065,-1.309,0.191,-0.214,0.043



=== RANDOM EFFECTS / VARIANCE-COVARIANCE ===


,Parameter,Coef.,Std.Err.
0,Group Var,0.854,0.600


In [ ]:
# flip test: gap = 2
gap_2 = sentence_df[sentence_df['gap_count'] == 2]
iv_list = ['wh_numeric']
mixed_effects_linear_regression(gap_2, iv_list, 'embed_surprisal', 'double gap embed: gap = 2')


=== DOUBLE GAP EMBED: GAP = 2 ===

=== MODEL SUMMARY ===
:            MixedLM Dependent Variable: embed_surprisal
No. Observations: 40
Method: REML
No. Groups: 20
Scale: 0.1216
Min. group size: 2
Log-Likelihood: -47.3003
Max. group size: 2
Converged: Yes
Mean group size: 2.0

=== FIXED EFFECTS COEFFICIENTS ===


,Parameter,Coef.,Std.Err.,z,P>|z|,[0.025,0.975]
0,Intercept,8.609,0.264,32.668,0.000,8.093,9.126
1,wh_numeric,-0.243,0.055,-4.413,0.000,-0.351,-0.135



=== RANDOM EFFECTS / VARIANCE-COVARIANCE ===


,Parameter,Coef.,Std.Err.
0,Group Var,1.328,1.828


In [ ]:
# division by grammaticality
iv_list = ['grammaticality']
mixed_effects_linear_regression(sentence_df, iv_list, 'embed_surprisal', 'double gap embed: division by grammaticality')


=== DOUBLE GAP EMBED: DIVISION BY GRAMMATICALITY ===

=== MODEL SUMMARY ===
Model: MixedLM
Dependent Variable: embed_surprisal
No. Observations: 160
Method: REML
No. Groups: 20
Scale: 0.5357
Min. group size: 8
Log-Likelihood: -204.7178
Max. group size: 8
Converged: Yes
Mean group size: 8.0

=== FIXED EFFECTS COEFFICIENTS ===


,Parameter,Coef.,Std.Err.,z,P>|z|,[0.025,0.975]
0,Intercept,7.319,0.234,31.335,0.000,6.861,7.777
1,grammaticality[T.gram-],0.647,0.120,5.415,0.000,0.413,0.881



=== RANDOM EFFECTS / VARIANCE-COVARIANCE ===


,Parameter,Coef.,Std.Err.
0,Group Var,0.912,0.463


### Islands

In [ ]:
def assign_grammaticality_islands(row, island_type):
    ''' Determines grammaticality based on licensor and gap values.'''
    if row[island_type] == 'null' or row[island_type] == 'that':
        if row['filler'] > 0 and row['gap'] == 1:
            return 'gram+'
            #return 1
        elif row['filler'] == 0 and row['gap'] == 0:
            return 'gram+'
            #return 1
        elif row['filler'] > 0 and row['gap'] == 0:
            return 'gram-'
            #return 0
        elif row['filler'] == 0 and row['gap'] == 1:
            return 'gram-'
            #return 0
    else:
        if row['filler'] > 0 and row['gap'] == 1:
            return 'gram-'
            #return 0
        elif row['filler'] == 0 and row['gap'] == 0:
            return 'gram+'
            #return 1
        elif row['filler'] > 0 and row['gap'] == 0:
            return 'gram-'
            #return 0
        elif row['filler'] == 0 and row['gap'] == 1:
            return 'gram-'
            #return 0

#### Wh-Islands

In [ ]:
sentence_df = pd.read_csv('test_sentences/Wh-Islands.csv')
sentence_df['comp_type'] = sentence_df['comp_type'].fillna('null')

In [ ]:
sentence_df[['cont', 'end']] = sentence_df['cont'].apply(split_ends)
sentence_df = encode_wh_licensor(sentence_df)
sentence_df['gap_numeric'] = sentence_df['gap'].replace(0, -1)
syntactic_parts = ['prefix', 'filler_word', 'subj1', 'verb1', 'comp', 'subj2', 'embed_verb', 'obj', 'cont']
sentence_df['sentence'] = sentence_df.apply(lambda x: merge_sentence (x, syntactic_parts), axis = 1)
sentence_df['surprisals'] = sentence_df['sentence'].apply(calculate_surprisal)
sentence_df['comp_type'] = pd.Categorical(sentence_df['comp_type'], categories = ['null', 'that', 'whether'], ordered = True)
sentence_df['grammaticality'] = sentence_df.apply(lambda x: assign_grammaticality_islands(x, 'comp_type'), axis = 1)

Post-Gap Region - Continuation

In [ ]:
sentence_df['region_surprisal'] = sentence_df.apply(lambda x: sum_region_surprisal(x, ['cont'], 'obj'), axis = 1)

In [ ]:
# flip test: gap-
no_gap_df = sentence_df[sentence_df['gap_numeric'] == -1]
iv_list = ['wh_numeric', 'comp_type']
mixed_effects_linear_regression(no_gap_df, iv_list, 'region_surprisal', 'wh-islands continuation: gap-')


=== WH-ISLANDS CONTINUATION: GAP- ===

=== MODEL SUMMARY ===
Model: MixedLM
Dependent Variable: region_surprisal
No. Observations: 120
Method: REML
No. Groups: 20
Scale: 0.7672
Min. group size: 6
Log-Likelihood: -211.5234
Max. group size: 6
Converged: Yes
Mean group size: 6.0

=== FIXED EFFECTS COEFFICIENTS ===


,Parameter,Coef.,Std.Err.,z,P>|z|,[0.025,0.975]
0,Intercept,18.655,1.362,13.697,0.000,15.985,21.324
1,comp_type[T.that],-1.100,0.196,-5.614,0.000,-1.483,-0.716
2,comp_type[T.whether],-1.374,0.196,-7.013,0.000,-1.757,-0.990
3,wh_numeric,1.120,0.138,8.089,0.000,0.849,1.392
4,wh_numeric:comp_type[T.that],-0.594,0.196,-3.031,0.002,-0.977,-0.210
5,wh_numeric:comp_type[T.whether],-0.967,0.196,-4.939,0.000,-1.351,-0.584



=== RANDOM EFFECTS / VARIANCE-COVARIANCE ===


,Parameter,Coef.,Std.Err.
0,Group Var,36.715,14.945


In [ ]:
# flip test: gap+
gap_df = sentence_df[sentence_df['gap_numeric'] == 1]
iv_list = ['wh_numeric', 'comp_type']
mixed_effects_linear_regression(gap_df, iv_list, 'region_surprisal', 'wh-islands continuation: gap+')


=== WH-ISLANDS CONTINUATION: GAP+ ===

=== MODEL SUMMARY ===
Model: MixedLM
Dependent Variable: region_surprisal
No. Observations: 120
Method: REML
No. Groups: 20
Scale: 1.1891
Min. group size: 6
Log-Likelihood: -242.1215
Max. group size: 6
Converged: Yes
Mean group size: 6.0

=== FIXED EFFECTS COEFFICIENTS ===


,Parameter,Coef.,Std.Err.,z,P>|z|,[0.025,0.975]
0,Intercept,25.806,2.276,11.339,0.000,21.346,30.267
1,comp_type[T.that],0.777,0.244,3.185,0.001,0.299,1.255
2,comp_type[T.whether],1.259,0.244,5.162,0.000,0.781,1.737
3,wh_numeric,-0.514,0.172,-2.980,0.003,-0.852,-0.176
4,wh_numeric:comp_type[T.that],0.018,0.244,0.074,0.941,-0.460,0.496
5,wh_numeric:comp_type[T.whether],0.413,0.244,1.694,0.090,-0.065,0.891



=== RANDOM EFFECTS / VARIANCE-COVARIANCE ===


,Parameter,Coef.,Std.Err.
0,Group Var,102.991,33.629


Embedded Clause

In [ ]:
embed = ['subj1', 'verb1', 'comp', 'subj2', 'embed_verb', 'obj', 'cont', 'end']
sentence_df['embed_surprisal'] = sentence_df.apply(lambda x: sum_region_surprisal(x, embed, normalize = True), axis = 1)

In [ ]:
# flip test: gap-
no_gap_df = sentence_df[sentence_df['gap_numeric'] == -1]
iv_list = ['wh_numeric', 'comp_type']
mixed_effects_linear_regression(no_gap_df, iv_list, 'embed_surprisal', 'wh-islands embed: gap-')


=== WH-ISLANDS EMBED: GAP- ===

=== MODEL SUMMARY ===
Model: MixedLM
Dependent Variable: embed_surprisal
No. Observations: 120
Method: REML
No. Groups: 20
Scale: 0.0600
Min. group size: 6
Log-Likelihood: -59.2084
Max. group size: 6
Converged: Yes
Mean group size: 6.0

=== FIXED EFFECTS COEFFICIENTS ===


,Parameter,Coef.,Std.Err.,z,P>|z|,[0.025,0.975]
0,Intercept,9.230,0.264,34.973,0.000,8.713,9.747
1,comp_type[T.that],-0.744,0.055,-13.580,0.000,-0.851,-0.636
2,comp_type[T.whether],-0.191,0.055,-3.485,0.000,-0.298,-0.084
3,wh_numeric,0.043,0.039,1.110,0.267,-0.033,0.119
4,wh_numeric:comp_type[T.that],0.078,0.055,1.422,0.155,-0.029,0.185
5,wh_numeric:comp_type[T.whether],-0.147,0.055,-2.681,0.007,-0.254,-0.039



=== RANDOM EFFECTS / VARIANCE-COVARIANCE ===


,Parameter,Coef.,Std.Err.
0,Group Var,1.363,1.993


In [ ]:
# flip test: gap+
gap_df = sentence_df[sentence_df['gap_numeric'] == 1]
iv_list = ['wh_numeric', 'comp_type']
mixed_effects_linear_regression(gap_df, iv_list, 'embed_surprisal', 'wh-islands embed: gap+')


=== WH-ISLANDS EMBED: GAP+ ===

=== MODEL SUMMARY ===
Model: MixedLM
Dependent Variable: embed_surprisal
No. Observations: 120
Method: REML
No. Groups: 20
Scale: 0.0902
Min. group size: 6
Log-Likelihood: -81.5676
Max. group size: 6
Converged: Yes
Mean group size: 6.0

=== FIXED EFFECTS COEFFICIENTS ===


,Parameter,Coef.,Std.Err.,z,P>|z|,[0.025,0.975]
0,Intercept,9.530,0.309,30.811,0.000,8.924,10.136
1,comp_type[T.that],-0.702,0.067,-10.455,0.000,-0.834,-0.570
2,comp_type[T.whether],0.012,0.067,0.184,0.854,-0.119,0.144
3,wh_numeric,-0.112,0.047,-2.354,0.019,-0.205,-0.019
4,wh_numeric:comp_type[T.that],0.158,0.067,2.355,0.019,0.027,0.290
5,wh_numeric:comp_type[T.whether],-0.049,0.067,-0.730,0.466,-0.181,0.083



=== RANDOM EFFECTS / VARIANCE-COVARIANCE ===


,Parameter,Coef.,Std.Err.
0,Group Var,1.868,2.229


In [ ]:
# division by grammaticality
iv_list = ['grammaticality']
mixed_effects_linear_regression(sentence_df, iv_list, 'embed_surprisal', 'wh-islands embed: division by grammaticality')


=== WH-ISLANDS EMBED: DIVISION BY GRAMMATICALITY ===

=== MODEL SUMMARY ===
Model: MixedLM
Dependent Variable: embed_surprisal
No. Observations: 240
Method: REML
No. Groups: 20
Scale: 0.2652
Min. group size: 12
Log-Likelihood: -225.1348
Max. group size: 12
Converged: Yes
Mean group size: 12.0

=== FIXED EFFECTS COEFFICIENTS ===


,Parameter,Coef.,Std.Err.,z,P>|z|,[0.025,0.975]
0,Intercept,8.998,0.284,31.630,0.000,8.440,9.555
1,grammaticality[T.gram-],0.191,0.067,2.840,0.005,0.059,0.324



=== RANDOM EFFECTS / VARIANCE-COVARIANCE ===


,Parameter,Coef.,Std.Err.
0,Group Var,1.565,1.042


#### Adjunct Islands 

In [ ]:
sentence_df = pd.read_csv('test_sentences/Adjunct Islands.csv')
sentence_df['adjunct_type'] = sentence_df['adjunct_type'].fillna('null')

In [ ]:
sentence_df[['cont', 'end']] = sentence_df['cont'].apply(split_ends)
sentence_df['adjunct_front_trigger'] = np.nan
sentence_df['adjunct_front_trigger'] = sentence_df['adjunct_trigger'].where(sentence_df['adjunct_type'] == 'front', sentence_df['adjunct_front_trigger'])
sentence_df['adjunct_trigger'] = sentence_df['adjunct_trigger'].where(sentence_df['adjunct_type'] != 'front', np.nan)
sentence_df.rename(columns = {'adjunct_trigger': 'adjunct_back_trigger'}, inplace = True)
sentence_df = sentence_df[[col for col in sentence_df.columns if col != 'end'] + ['end']]

In [ ]:
sentence_df = encode_wh_licensor(sentence_df)
sentence_df['gap_numeric'] = sentence_df['gap'].replace(0, -1)
syntactic_parts = ['prefix', 'filler_word', 'adjunct_back_trigger', 'prep', 'subj', 'mod', 'verb', 'obj', 'cont', 'adjunct_front_trigger']
sentence_df['sentence'] = sentence_df.apply(lambda x: merge_sentence(x, syntactic_parts), axis = 1)
sentence_df['surprisals'] = sentence_df['sentence'].apply(calculate_surprisal)
sentence_df['adjunct_type'] = pd.Categorical(sentence_df['adjunct_type'], categories = ['null', 'back', 'front'], ordered = True)
sentence_df['grammaticality'] = sentence_df.apply(lambda x: assign_grammaticality_islands(x, 'adjunct_type'), axis = 1)

Post-Gap Region - Continuation

In [ ]:
sentence_df['region_surprisal'] = sentence_df.apply(lambda x: sum_region_surprisal(x, ['cont'], 'obj'), axis = 1)

In [ ]:
# flip test: gap-
no_gap_df = sentence_df[sentence_df['gap_numeric'] == -1]
iv_list = ['wh_numeric', 'adjunct_type']
mixed_effects_linear_regression(no_gap_df, iv_list, 'region_surprisal', 'adjunct islands continuation: gap-')


=== ADJUNCT ISLANDS CONTINUATION: GAP- ===

=== MODEL SUMMARY ===
Model: MixedLM
Dependent Variable: region_surprisal
No. Observations: 120
Method: REML
No. Groups: 20
Scale: 0.6131
Min. group size: 6
Log-Likelihood: -209.8872
Max. group size: 6
Converged: Yes
Mean group size: 6.0

=== FIXED EFFECTS COEFFICIENTS ===


,Parameter,Coef.,Std.Err.,z,P>|z|,[0.025,0.975]
0,Intercept,27.740,2.184,12.703,0.000,23.460,32.020
1,adjunct_type[T.back],-1.370,0.175,-7.826,0.000,-1.713,-1.027
2,adjunct_type[T.front],-1.228,0.175,-7.016,0.000,-1.572,-0.885
3,wh_numeric,1.050,0.124,8.483,0.000,0.808,1.293
4,wh_numeric:adjunct_type[T.back],-0.913,0.175,-5.215,0.000,-1.256,-0.570
5,wh_numeric:adjunct_type[T.front],-0.725,0.175,-4.140,0.000,-1.068,-0.382



=== RANDOM EFFECTS / VARIANCE-COVARIANCE ===


,Parameter,Coef.,Std.Err.
0,Group Var,95.065,43.196


In [ ]:
# flip test: gap+
gap_df = sentence_df[sentence_df['gap_numeric'] == 1]
iv_list = ['wh_numeric', 'adjunct_type']
mixed_effects_linear_regression(gap_df, iv_list, 'region_surprisal', 'adjunct islands continuation: gap+')


=== ADJUNCT ISLANDS CONTINUATION: GAP+ ===

=== MODEL SUMMARY ===
Model: MixedLM
Dependent Variable: region_surprisal
No. Observations: 120
Method: REML
No. Groups: 20
Scale: 0.9119
Min. group size: 6
Log-Likelihood: -223.9159
Max. group size: 6
Converged: Yes
Mean group size: 6.0

=== FIXED EFFECTS COEFFICIENTS ===


,Parameter,Coef.,Std.Err.,z,P>|z|,[0.025,0.975]
0,Intercept,35.380,1.696,20.857,0.000,32.055,38.705
1,adjunct_type[T.back],-0.279,0.214,-1.305,0.192,-0.697,0.140
2,adjunct_type[T.front],-0.000,0.214,-0.002,0.998,-0.419,0.418
3,wh_numeric,0.203,0.151,1.345,0.179,-0.093,0.499
4,wh_numeric:adjunct_type[T.back],-0.070,0.214,-0.329,0.742,-0.489,0.348
5,wh_numeric:adjunct_type[T.front],-0.151,0.214,-0.706,0.480,-0.569,0.268



=== RANDOM EFFECTS / VARIANCE-COVARIANCE ===


,Parameter,Coef.,Std.Err.
0,Group Var,57.094,21.306


Embedded Clause

In [ ]:
embed = ['adjunct_back_trigger', 'prep', 'subj', 'mod', 'verb', 'obj', 'cont', 'adjunct_front_trigger', 'end']
sentence_df['embed_surprisal'] = sentence_df.apply(lambda x: sum_region_surprisal(x, embed, normalize = True), axis = 1)

In [ ]:
# flip test: gap-
no_gap_df = sentence_df[sentence_df['gap_numeric'] == -1]
iv_list = ['wh_numeric', 'adjunct_type']
mixed_effects_linear_regression(no_gap_df, iv_list, 'embed_surprisal', 'adjunct islands embed: gap-')


=== ADJUNCT ISLANDS EMBED: GAP- ===

=== MODEL SUMMARY ===
Model: MixedLM
Dependent Variable: embed_surprisal
No. Observations: 120
Method: REML
No. Groups: 20
Scale: 0.0749
Min. group size: 6
Log-Likelihood: -69.2111
Max. group size: 6
Converged: Yes
Mean group size: 6.0

=== FIXED EFFECTS COEFFICIENTS ===


,Parameter,Coef.,Std.Err.,z,P>|z|,[0.025,0.975]
0,Intercept,8.578,0.257,33.381,0.000,8.074,9.081
1,adjunct_type[T.back],0.014,0.061,0.221,0.825,-0.106,0.133
2,adjunct_type[T.front],0.546,0.061,8.921,0.000,0.426,0.666
3,wh_numeric,0.294,0.043,6.792,0.000,0.209,0.379
4,wh_numeric:adjunct_type[T.back],-0.222,0.061,-3.626,0.000,-0.342,-0.102
5,wh_numeric:adjunct_type[T.front],-0.060,0.061,-0.989,0.323,-0.180,0.059



=== RANDOM EFFECTS / VARIANCE-COVARIANCE ===


,Parameter,Coef.,Std.Err.
0,Group Var,1.283,1.682


In [ ]:
# flip test: gap+
gap_df = sentence_df[sentence_df['gap_numeric'] == 1]
iv_list = ['wh_numeric', 'adjunct_type']
mixed_effects_linear_regression(gap_df, iv_list, 'embed_surprisal', 'adjunct islands embed: gap+')


=== ADJUNCT ISLANDS EMBED: GAP+ ===

=== MODEL SUMMARY ===
Model: MixedLM
Dependent Variable: embed_surprisal
No. Observations: 120
Method: REML
No. Groups: 20
Scale: 0.0697
Min. group size: 6
Log-Likelihood: -62.9370
Max. group size: 6
Converged: Yes
Mean group size: 6.0

=== FIXED EFFECTS COEFFICIENTS ===


,Parameter,Coef.,Std.Err.,z,P>|z|,[0.025,0.975]
0,Intercept,8.252,0.222,37.181,0.000,7.817,8.687
1,adjunct_type[T.back],0.178,0.059,3.017,0.003,0.062,0.294
2,adjunct_type[T.front],0.771,0.059,13.068,0.000,0.656,0.887
3,wh_numeric,0.243,0.042,5.834,0.000,0.162,0.325
4,wh_numeric:adjunct_type[T.back],-0.168,0.059,-2.844,0.004,-0.284,-0.052
5,wh_numeric:adjunct_type[T.front],0.002,0.059,0.028,0.978,-0.114,0.117



=== RANDOM EFFECTS / VARIANCE-COVARIANCE ===


,Parameter,Coef.,Std.Err.
0,Group Var,0.950,1.295


In [ ]:
# division by grammaticality
iv_list = ['grammaticality']
mixed_effects_linear_regression(sentence_df, iv_list, 'embed_surprisal', 'adjunct islands embed: division by grammaticality')


=== ADJUNCT ISLANDS EMBED: DIVISION BY GRAMMATICALITY ===

=== MODEL SUMMARY ===
Model: MixedLM
Dependent Variable: embed_surprisal
No. Observations: 240
Method: REML
No. Groups: 20
Scale: 0.2691
Min. group size: 12
Log-Likelihood: -222.9743
Max. group size: 12
Converged: Yes
Mean group size: 12.0

=== FIXED EFFECTS COEFFICIENTS ===


,Parameter,Coef.,Std.Err.,z,P>|z|,[0.025,0.975]
0,Intercept,8.547,0.236,36.169,0.000,8.084,9.010
1,grammaticality[T.gram-],0.179,0.071,2.515,0.012,0.039,0.318



=== RANDOM EFFECTS / VARIANCE-COVARIANCE ===


,Parameter,Coef.,Std.Err.
0,Group Var,1.050,0.699


#### Complex NP Islands

In [ ]:
sentence_df = pd.read_csv('test_sentences/Complex NP Islands.csv')

In [ ]:
sentence_df = encode_wh_licensor(sentence_df)
sentence_df['gap_numeric'] = sentence_df['gap'].replace(0, -1)
sentence_df['sentence'] = sentence_df.apply(merge_sentence, axis = 1)
sentence_df['surprisals'] = sentence_df['sentence'].apply(calculate_surprisal)
sentence_df['grammaticality'] = sentence_df.apply(assign_grammaticality_islands, axis = 1)

Subject Condition

In [ ]:
subject_df = sentence_df.copy()

In [ ]:
subject_df['region_surprisal'] = subject_df.apply(lambda x: sum_region_surprisal(x, ['subj_setup']), axis = 1)
subject_df

In [ ]:
# filter for subject condition
subject_df = subject_df[subject_df['subj_obj'] == 'subject']

In [ ]:
# flip test: gap-
no_gap_df = subject_df[subject_df['gap_numeric'] == -1]
iv_list = ['wh_numeric']
mixed_effects_linear_regression(no_gap_df, iv_list, 'region_surprisal', 'complex np islands subject condition: gap-')

In [ ]:
# flip test: gap+
gap_df = subject_df[subject_df['gap_numeric'] == 1]
iv_list = ['wh_numeric']
mixed_effects_linear_regression(gap_df, iv_list, 'region_surprisal', 'complex np islands subject condition: gap+')

In [ ]:
# division by grammaticality
iv_list = ['grammaticality']
mixed_effects_linear_regression(subject_df, iv_list, 'region_surprisal', 'complex np islands subject condition: division by grammaticality')

Object Condition

In [ ]:
object_df = sentence_df.copy()

In [ ]:
object_df['region_surprisal'] = object_df.apply(lambda x: sum_region_surprisal(x, ['end']), axis = 1)
object_df

In [ ]:
# filter for object gap
object_df = object_df[object_df['subj_obj'] == 'object']

In [ ]:
# flip test: gap-
no_gap_df = object_df[object_df['gap_numeric'] == -1]
iv_list = ['wh_numeric']
mixed_effects_linear_regression(no_gap_df, iv_list, 'region_surprisal', 'complex np islands object condition: gap-')

In [ ]:
# flip test: gap+
gap_df = object_df[object_df['gap_numeric'] == 1]
iv_list = ['wh_numeric']
mixed_effects_linear_regression(gap_df, iv_list, 'region_surprisal', 'complex np islands object condition: gap+')

In [ ]:
# division by grammaticality
iv_list = ['grammaticality']
mixed_effects_linear_regression(object_df, iv_list, 'region_surprisal', 'complex np islands object condition: division by grammaticality')

Embedded Clause

In [ ]:
embed = ['subj', 'that_rc', 'what_rc', 'rc_np', 'prep', 'prep_np', 'subj_setup', 'obj_setup', 'end']
sentence_df['embed_surprisal'] = sentence_df.apply(lambda x: sum_region_surprisal(x, embed, normalize = True), axis = 1)
sentence_df

Embedded Clause - Subject Condition

In [ ]:
# filter for subject condition
subject_df = sentence_df[sentence_df['subj_obj'] == 'subject']

In [ ]:
# flip test: gap-
no_gap_df = subject_df[subject_df['gap_numeric'] == -1]
iv_list = ['wh_numeric']
mixed_effects_linear_regression(no_gap_df, iv_list, 'embed_surprisal', 'complex np islands subject condition: gap-')

In [ ]:
# flip test: gap+
gap_df = subject_df[subject_df['gap_numeric'] == 1]
iv_list = ['wh_numeric']
mixed_effects_linear_regression(gap_df, iv_list, 'embed_surprisal', 'complex np islands subject condition: gap+')

In [ ]:
# division by grammaticality
iv_list = ['grammaticality']
mixed_effects_linear_regression(subject_df, iv_list, 'embed_surprisal', 'complex np islands subject condition: division by grammaticality')

Embedded Clause - Object Condition

In [ ]:
# filter for object gap
object_df = sentence_df[sentence_df['subj_obj'] == 'object']

In [ ]:
# flip test: gap-
no_gap_df = object_df[object_df['gap_numeric'] == -1]
iv_list = ['wh_numeric']
mixed_effects_linear_regression(no_gap_df, iv_list, 'embed_surprisal', 'complex np islands object condition: gap-')

In [ ]:
# flip test: gap+
gap_df = object_df[object_df['gap_numeric'] == 1]
iv_list = ['wh_numeric']
mixed_effects_linear_regression(gap_df, iv_list, 'embed_surprisal', 'complex np islands object condition: gap+')

In [ ]:
# division by grammaticality
iv_list = ['grammaticality']
mixed_effects_linear_regression(object_df, iv_list, 'embed_surprisal', 'complex np islands object condition: division by grammaticality')